In [10]:
import pandas as pd
import os
import numpy as np
#from six import b
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf

"""
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D , BatchNormalization , Dropout , Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint , LearningRateScheduler
"""

'\nfrom tensorflow.keras.applications.resnet50 import ResNet50\nfrom tensorflow.keras.layers import GlobalAveragePooling2D , BatchNormalization , Dropout , Dense\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint , LearningRateScheduler\n'

In [2]:
HOME_TRAIN = False

In [3]:
BATCH_SIZE = 32

In [4]:
dataset_info = pd.read_csv("meta_data_for_train_K-Face.csv")
dataset_info

,file_path,ID,연령대,성별
0,f:/High_Resolution\19062421\S001\L1\E01\C10.jpg,19062421,30대,남
1,f:/High_Resolution\19062421\S001\L1\E01\C19.jpg,19062421,30대,남
2,f:/High_Resolution\19062421\S001\L1\E01\C20.jpg,19062421,30대,남
3,f:/High_Resolution\19062421\S001\L1\E01\C4.jpg,19062421,30대,남
4,f:/High_Resolution\19062421\S001\L1\E01\C5.jpg,19062421,30대,남
...,...,...,...,...
583195,f:/High_Resolution\19101513\S003\L9\E02\C5.jpg,19101513,20대,여
583196,f:/High_Resolution\19101513\S003\L9\E02\C6.jpg,19101513,20대,여
583197,f:/High_Resolution\19101513\S003\L9\E02\C7.jpg,19101513,20대,여
583198,f:/High_Resolution\19101513\S003\L9\E02\C8.jpg,19101513,20대,여


In [5]:
def gender_map( gender ):
    if gender == '남':
        return 1
    elif gender == '여':
        return 0

In [6]:
dataset_info['new_gender'] = dataset_info['성별'].map( gender_map )
dataset_info.head()

,file_path,ID,연령대,성별,new_gender
0,f:/High_Resolution\19062421\S001\L1\E01\C10.jpg,19062421,30대,남,1
1,f:/High_Resolution\19062421\S001\L1\E01\C19.jpg,19062421,30대,남,1
2,f:/High_Resolution\19062421\S001\L1\E01\C20.jpg,19062421,30대,남,1
3,f:/High_Resolution\19062421\S001\L1\E01\C4.jpg,19062421,30대,남,1
4,f:/High_Resolution\19062421\S001\L1\E01\C5.jpg,19062421,30대,남,1


In [7]:
data_file_path = dataset_info['file_path'].tolist()
gender = dataset_info['new_gender'].tolist()
#age = dataset_info['age'].tolist()

In [8]:
if HOME_TRAIN == False:
    data_file_path = [c.replace('f:/', 'C:/Users/csyi/Desktop/Age_Gender_Prediction/Dataset/K-Face/') for c in data_file_path]

In [11]:
le_gender = LabelEncoder( ).fit_transform(gender)

gender = tf.keras.utils.to_categorical(le_gender, num_classes=2)
gender

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [12]:
file_path_train, file_path_val, y_train, y_val = train_test_split(data_file_path, gender, 
                                                                  test_size=0.25, 
                                                                  random_state=777, 
                                                                  stratify = gender)

In [13]:
print( len(file_path_train) , len(y_train) , len(file_path_val) , len(y_val) )

437400 437400 145800 145800


In [14]:
def load_image( image_path , label ):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.resnet50.preprocess_input(img)
    return img , label

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices( (file_path_train , y_train) )
val_dataset = tf.data.Dataset.from_tensor_slices( (file_path_val , y_val) )

In [16]:
train_dataset = train_dataset.shuffle(buffer_size=len(file_path_train))\
                .map( load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                .batch(BATCH_SIZE)\
                .prefetch(tf.data.experimental.AUTOTUNE)     #

val_dataset = val_dataset.shuffle(buffer_size=len(file_path_val))\
                .map( load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                .batch(BATCH_SIZE)\
                .prefetch(tf.data.experimental.AUTOTUNE)    #

In [17]:
train_dataset.take(1)

<TakeDataset shapes: ((None, 224, 224, 3), (None, 2)), types: (tf.float32, tf.float32)>